# 🚀 Advanced RVC Inference Pro

**State-of-the-art Voice Conversion with KADVC Optimization**

> **Single Source of Truth (SSOT) Colab Notebook** - Optimized with uv for 10x faster installation

This notebook provides:
- ⚡ **uv Package Manager** - 10x faster dependency installation
- 🗄️ **Drive Mounting** - Auto symlink weights folder
- 🌐 **Tunneling** - Robust Gradio/ngrok integration  
- 🎯 **GPU Auto-detection** - Tesla T4/P100/A100 optimization
- 🧠 **Memory Management** - Automatic OOM prevention
- 🎵 **Voice Processing** - High-quality RVC voice conversion
- 🔧 **Model Loading** - Automatic model detection and loading

**Repository:** [ArkanDash/Advanced-RVC-Inference](https://github.com/ArkanDash/Advanced-RVC-Inference)

**Author:** BF667

# Colab Parameters

<!-- This cell contains parameters for the notebook. These parameters should not be modified. -->
<!-- Parameters specific to Advanced RVC Inference -->


In [ ]:
# @title 📦 Install Dependencies with uv (10x Faster)
cuda_version = ""  # e.g., "11.8" or "12.1", leave empty for auto#@param {type: "string"}

import os
import sys
import time
import subprocess
import json
from pathlib import Path

# Check if running in Colab
try:
    import google.colab
    IN_COLAB = True
    print("🚀 Running in Google Colab environment")
except ImportError:
    IN_COLAB = False
    print("⚠️ Not running in Colab - some features may not work")

# Setup paths
BASE_DIR = Path.cwd()
CACHE_FILE = BASE_DIR / ".colab_deps_cache.json"

# Install CUDA toolkit if specified
if cuda_version and IN_COLAB:
    print(f"🖥️ Installing CUDA {cuda_version} toolkit...")
    subprocess.run(["apt", "update"], check=True)
    subprocess.run(["apt", "install", "-y", f"cuda-toolkit-{cuda_version}"], check=True)

# Define package requirements
REQUIREMENTS = [
    "torch>=2.0.0",
    "torchvision",
    "torchaudio",
    "librosa",
    "soundfile",
    "numpy",
    "scipy",
    "gradio>=3.50.0",
    "transformers",
    "huggingface-hub",
    "omegaconf",
    "numba",
    "ffmpeg-python",
    "audioread",
    "resampy",
    "parselmouth",
]

# Cache management
def load_cache():
    if CACHE_FILE.exists():
        with open(CACHE_FILE, "r") as f:
            return json.load(f)
    return {}

def save_cache(cache):
    with open(CACHE_FILE, "w") as f:
        json.dump(cache, f, indent=2)

# Check if packages are already installed
def check_package_installed(package_name):
    try:
        result = subprocess.run(
            ["pip", "show", package_name.split(">=")[0].split("<")[0].split("==")[0]],
            capture_output=True,
            text=True,
            timeout=10
        )
        return result.returncode == 0
    except (subprocess.TimeoutExpired, subprocess.SubprocessError):
        return False

# Main installation logic
print("🔍 Checking existing package installation...")
cache = load_cache()
start_time = time.time()

# Install packages that are not already installed
packages_to_install = []
for req in REQUIREMENTS:
    package_name = req.split(">=")[0].split("<")[0].split("==")[0]
    if not check_package_installed(package_name):
        packages_to_install.append(req)
        print(f"  ➕ Will install: {req}")
    else:
        print(f"  ✅ Already installed: {package_name}")

if packages_to_install:
    print(f"\n📦 Installing {len(packages_to_install)} packages...")
    
    # Install uv first for blazing fast package management (Following Colab best practices)
    print("⚡ Installing uv package manager for 10x faster installation...")
    subprocess.run(["pip", "install", "uv"], check=True)
    
    # Install PyTorch first (important for CUDA)
    torch_reqs = [req for req in packages_to_install if "torch" in req.lower()]
    other_reqs = [req for req in packages_to_install if "torch" not in req.lower()]
    
    if torch_reqs:
        print("🔥 Installing PyTorch components with uv (ultra-fast)...")
        for req in torch_reqs:
            subprocess.run(["uv", "pip", "install", req, "--index-url", "https://download.pytorch.org/whl/cu118"], check=True)
    
    if other_reqs:
        print("📚 Installing other dependencies with uv (lightning fast)...")
        # Install multiple packages at once for maximum speed
        subprocess.run(["uv", "pip", "install"] + other_reqs, check=True)
        
    print("✅ All packages installed successfully!")
else:
    print("\n🎉 All required packages are already installed!")

# Verify critical packages
critical_packages = ["torch", "gradio", "librosa"]
print("\n🔍 Verifying critical packages...")
for pkg in critical_packages:
    if check_package_installed(pkg):
        print(f"  ✅ {pkg}")
    else:
        print(f"  ❌ {pkg} - Installation failed!")

installation_time = time.time() - start_time
print(f"\n⏱️ Total installation time: {installation_time:.2f} seconds")
print("\n🎉 Dependency installation completed with uv optimization!")


# @title ## 🗄️ Mount Google Drive & Setup Symlinks

Configure persistent storage for your RVC models following Colab best practices.

In [ ]:
# @title Setup Google Drive Storage
rvc_dir_name = "Advanced-RVC-Models"  # Default folder name on Drive#@param {type: "string"}
backup_existing = "no"  # Options: "yes" or "no"#@param ["yes", "no"]

from google.colab import drive
import os
from pathlib import Path
import shutil
import subprocess

print("🔗 Mounting Google Drive...")
drive.mount('/content/drive')

# Define paths
drive_base = Path("/content/drive/MyDrive")
local_weights = Path("/content/Advanced-RVC-Inference/weights")
local_models = Path("/content/Advanced-RVC-Inference/models")
drive_rvc_dir = drive_base / rvc_dir_name

print(f"📁 Drive directory: {drive_rvc_dir}")
print(f"📂 Local weights: {local_weights}")

# Create Drive directory if it doesn't exist
drive_rvc_dir.mkdir(exist_ok=True)

# Handle existing models backup
if backup_existing.lower() == "yes":
    if drive_rvc_dir.exists() and any(drive_rvc_dir.iterdir()):
        backup_path = drive_rvc_dir / f"backup_{int(time.time())}"
        print(f"🔄 Backing up existing models to: {backup_path}")
        shutil.move(str(drive_rvc_dir), str(backup_path))
        drive_rvc_dir.mkdir(exist_ok=True)

# Create symlinks if they don't exist
print("🔗 Creating symlinks...")

if local_weights.exists():
    weights_link = drive_rvc_dir / "weights"
    if weights_link.exists():
        weights_link.unlink()
    weights_link.symlink_to(local_weights)
    print(f"  ✅ Weights symlinked: {weights_link}")

if local_models.exists():
    models_link = drive_rvc_dir / "models"
    if models_link.exists():
        models_link.unlink()
    models_link.symlink_to(local_models)
    print(f"  ✅ Models symlinked: {models_link}")

print("\n🎉 Drive setup completed successfully!")


# @title ## 📁 Project Setup

Clone repository and install project dependencies with uv optimization.

In [ ]:
# @title Clone and Setup Repository
repo_url = ""  #  Leave empty for default repository#@param {type: "string"}
branch = ""  # Leave empty for default branch#@param {type: "string"}

import os
import subprocess
from pathlib import Path

# Set defaults (Following Colab best practices for parameter handling)
if not repo_url:
    REPO_URL = "https://github.com/ArkanDash/Advanced-RVC-Inference.git"
else:
    REPO_URL = repo_url

if not branch:
    REPO_BRANCH = "master"
else:
    REPO_BRANCH = branch

PROJECT_DIR = Path.cwd() / "Advanced-RVC-Inference"

if not PROJECT_DIR.exists():
    print("📥 Cloning Advanced RVC Inference repository...")
    print(f"🌐 URL: {REPO_URL}")
    print(f"🌿 Branch: {REPO_BRANCH}")
    subprocess.run(["git", "clone", "-b", REPO_BRANCH, REPO_URL, str(PROJECT_DIR)], check=True)
    print("✅ Repository cloned successfully")
else:
    print("📁 Repository already exists, updating...")
    os.chdir(PROJECT_DIR)
    # Use the correct branch (master) - follows repository convention
    subprocess.run(["git", "pull", "origin", "master"], check=True)
    print("✅ Repository updated")

# Change to project directory
os.chdir(PROJECT_DIR)
print(f"📂 Working directory: {PROJECT_DIR}")

# Install project requirements with uv (faster installation following Colab best practices)
print("📦 Installing project requirements with uv (much faster)...")
if (PROJECT_DIR / "requirements.txt").exists():
    subprocess.run(["uv", "pip", "install", "-r", "requirements.txt"], check=True)
    print("✅ Requirements installed successfully")
else:
    print("⚠️ No requirements.txt found, skipping requirements installation")

# Set up environment variables
os.environ["PYTHONPATH"] = str(PROJECT_DIR)
print("🌍 Python path configured")


# @title ## 🌐 Setup Tunneling (Choose One)

Configure tunneling service for accessing the RVC interface.

In [ ]:
# @title Configure Tunneling Options
tunnel_service = "gradio"  #  Options: "ngrok" or "gradio"#@param ["ngrok", "gradio"]
ngrok_token = ""  #  Get free token from https://ngrok.com#@param {type: "string"}
gradio_port = 0  # 0 = auto-detect#@param {type: "number"}

import os
import subprocess
import time
from pathlib import Path

if tunnel_service == "ngrok" and ngrok_token:
    print("🔧 Setting up ngrok tunneling...")
    
    # Download and install ngrok
    if not Path("./ngrok").exists():
        print("📥 Downloading ngrok...")
        subprocess.run(["wget", "https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz"], check=True)
        subprocess.run(["tar", "-xzf", "ngrok-v3-stable-linux-amd64.tgz"], check=True)
        print("✅ ngrok downloaded and extracted")
    
    # Configure ngrok auth token
    subprocess.run(["./ngrok", "config", "add-authtoken", ngrok_token], check=True)
    print("✅ ngrok auth token configured")
    
elif tunnel_service == "gradio":
    print("🌐 Using built-in Gradio tunneling...")
    print("✅ Gradio tunneling requires no additional setup")
else:
    print("⚠️ Please configure ngrok token if using ngrok service")

print(f"🔌 Tunneling service configured: {tunnel_service}")


# @title ## 🚀 Launch Advanced RVC Interface

Start the RVC inference interface with optimized settings.

In [ ]:
# @title Launch RVC Interface
debug_mode = False#@param {type: "boolean"}
auto_open_browser = False  # not recommended in Colab#@param {type: "boolean"}

import os
import subprocess
import time
import threading
from pathlib import Path

# Set environment variables
os.environ["PYTHONPATH"] = str(Path.cwd() / "Advanced-RVC-Inference")

if debug_mode:
    os.environ["RVC_DEBUG"] = "1"
    print("🐛 Debug mode enabled")

print("🚀 Launching Advanced RVC Inference interface...")

# Change to project directory
project_dir = Path("Advanced-RVC-Inference")
os.chdir(project_dir)

# Start the application
print("🌟 Starting RVC Inference Pro...")

if tunnel_service == "ngrok":
    # Launch with ngrok
    cmd = [
        "python", "app.py",
        "--tunnel", "ngrok",
        "--ngrok-token", ngrok_token,
        "--debug" if debug_mode else ""
    ]
else:
    # Launch with Gradio
    port_arg = f"--port {gradio_port}" if gradio_port > 0 else ""
    cmd = [
        "python", "app.py",
        "--tunnel", "gradio",
        port_arg,
        "--debug" if debug_mode else ""
    ]

# Remove empty strings from command
cmd = [arg for arg in cmd if arg]

print(f"🔧 Command: {' '.join(cmd)}")
print("⏳ Starting application...")

# Start the application in a subprocess
try:
    if auto_open_browser and tunnel_service == "gradio":
        # Launch with browser auto-open (only for gradio)
        subprocess.run(cmd + ["--auto-open"], check=True)
    else:
        subprocess.run(cmd, check=True)
except KeyboardInterrupt:
    print("\n🛑 Application stopped by user")
except subprocess.CalledProcessError as e:
    print(f"❌ Error starting application: {e}")
    print("🔧 Please check the logs above for more details")
